Some general purpose scripts.
The code here is very ugly with extensive use of global variables. Do __not__ learn from it :)

# Imports and general routines

In [1]:
%pylab

Using matplotlib backend: Qt4Agg
Populating the interactive namespace from numpy and matplotlib


In [ ]:
%matplotlib inline

In [ ]:
import sys
import matplotlib
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
from scipy import stats
from scipy.optimize import curve_fit
from numpy.polynomial import polynomial 

In [ ]:
from sklearn import linear_model
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline

In [ ]:
import itertools

In [ ]:
import matplotlib.animation as animation
from IPython.display import HTML

In [ ]:
from __future__ import division

In [ ]:
%run  'savitzky_golay.ipynb'

In [ ]:
_my = sys.modules[__name__]

# Timing and plotting time of functions

In [ ]:
def is_iterable(obj):
    '''check if obj is iterable'''
    try:
        _ = iter(obj)
        return 1
    except TypeError:
        return 0 

In [ ]:
def inputlen(input):
    '''return some number that can be considered as the length of the input'''
    if not input:
        return 0
    if isinstance(input, basestring):
        return len(input)
    if isinstance( input, ( int, long ) ):
        return input
    if is_iterable(input):
        return sum([inputlen(i) for i in input])
    return len(repr(input))

In [ ]:
# some global variables
program_step_counter = 0

# these are temporary global variables needed due to quirks of %timeit magic
temp_func = None
temp_inputs = None
temp_output = None

In [ ]:
import itertools
def compare_times(func1,func2,gen,inlen=inputlen):
    gen1, gen2 = walk, walk2 = itertools.tee(gen)
    (_,_,lengths,times1,_) = timer(func1,gen1,doplot=False,doplot_logscale=False,interpolate=False)
    (_,_,lengths,times2,_) = timer(func2,gen2,doplot=False,doplot_logscale=False,interpolate=False)
    
    fig, ax1 = plt.subplots()
    lns1 = ax1.plot(lengths, times1, 'g-',label=func1.__name__)
    lns2 = ax1.plot(lengths, times2, 'b-',label=func2.__name__)
    lns = lns1+lns2
    labs = [l.get_label() for l in lns]
    ax1.legend(lns, labs, loc=0)

    ax1.set_xlabel('Input length')
    ax1.set_ylabel(r'Time ($\mu$s)', color='g')
    plt.show()

In [ ]:
def timer(func,gen,inlen=inputlen,doplot=True,doplot_logscale=True, smoothing_filter=False,interpolate=True):
    '''time a function on different inputs and plot it
       func: function to run
       gen:  list/tuple/iterable of inputs
       inlen: function to compute the input length (default: inlen)
       doplot: print a plot or not (default: True)
       
       returns a tuple of equal lengthed lists as follows:
       outputs: a list of the outputs
       curve: a function f(n) that estimates the number of steps for input length n
       lengths: a list of the input lengths
       times: a list of the times (in seconds)
       steps: a list of the number of steps (calls to stepit() ) that it took
    
    '''
    
    global program_step_counter
    global temp_func
    global temp_output
    
    lengths = []
    times = []
    steps = []
    outputs = []
    step_no = 0
    for inputs in gen:
        # sys.stdout.write("Running for input length "+ str(inlen(inputs))+"..")
        sys.stdout.write('.')
        sys.stdout.flush()
        step_no += 1
        program_step_counter = 0
        
        def testf():
            ans = None
            global temp_output
            if is_iterable(inputs) and not isinstance(inputs, basestring):
                ans = func(*inputs)
            else:
                ans = func(inputs)
            temp_output = ans
    
        temp_func = testf
        temp_output = None
        res = %timeit -n3 -r1 -o -q temp_func()
        if res.best < 10**(-3):
            time = "{0:0.0f}".format(res.best*(10**9))+ " nanoseconds"
        elif res.best < 1: 
            time = "{0:0.1f}".format(res.best*(10**3))+ " milliseconds"
        else: 
            time = "{0:0.2f}".format(res.best*(10**3))+ " seconds"
        if program_step_counter:
            time += "({0:g} steps)".format(program_step_counter)
        # print time
        outputs.append(temp_output)
        lengths.append(inlen(inputs))
        times.append(res.best)
        steps.append(int(program_step_counter/3))
    
    curve = plot_curve(lengths,times,steps,doplot,doplot_logscale,smoothing_filter,interpolate)
        
    return (outputs,curve,lengths,times,steps)
    
    

In [ ]:
def step_pc(inc_step=1):
    '''Increment the global counter of number of steps'''
    global program_step_counter
    # sys.stdout.write('*')
    program_step_counter += inc_step

In [ ]:
def integer_hist(map, n = None):
    '''histogram of an array/list of integers'''
    if not n:
        n = max(map)
    plt.hist(map,bins=np.linspace(0,n,n+1))
    plt.xlabel('Value')
    plt.ylabel('Frequency')
    plt.show()

In [ ]:
def poly_and_log(lengths,a,b,c):
    '''compute a function of the form f(n)=a*n^b*(log n)^c where c is effectively in {0,1}'''
    # print 'n,a,b,c:',n,a,b,c
    c = max(0,1-(c-1)*(c-1))
    print "n=",n
    ans = np.multiply(a*np.power(n,b),np.power(np.log2(n),c))
    print "ans=",ans
    return ans

In [ ]:
def mylog2(n):
    return math.log(max(n,2),2)

In [ ]:
def myround(x):
    return round(5*x)/5

def fit_poly_log(lengths,steps,smoothing_filter=False, allow_logs = True):
    """Fit to the steps vs lengths curve a function that has the form a*n^b(log n)^c 
       for some very specific choices of a,b,c
    """
    # print "fitting lengths, steps:", len(lengths),len(steps),lengths[:5],steps[:5]
    if allow_logs:
        funcs = [(lambda n:n**3),(lambda n: (n**2)*mylog2(n)),(lambda n: n**2),(lambda n: n*mylog2(n)),
                     (lambda n: n),(lambda n: mylog2(n))]
        descs = [r"n^3",r"n^2\log n",r"n^2",r"n\log n",r"n",r"\log n"]
    else:
        funcs = [(lambda n:n**3),(lambda n: n**2), (lambda n: n)]
        descs = [r"n^3",r"n^2",r"n"]
        
    
    features = [[funcs[i](n) for i in range(len(funcs))] for n in lengths]
    # print "Features:", features
    features = np.reshape(features,(-1,len(funcs)))
    if smoothing_filter:
        window_size = int(len(steps)/10)
        window_size = window_size + (window_size % 2) - 1
        if window_size >= 5:
            steps = savitzky_golay(steps, window_size, 3)
    
    steps = np.reshape(steps,(-1,1))
    
    # print "Features, steps", features[:5],steps[:5]
    #clf = linear_model.LinearRegression()
    clf = linear_model.LassoLars(max_iter=10000,alpha = 1.0, fit_intercept=False, positive=True)
    #clf = linear_model.RANSACRegressor(clf,residual_threshold=np.max(steps))
    clf.fit (features,steps)
    # return map(lambda c: round(c) if abs(c-round(c))<0.2  c,cls.coefs_)
    # print "Found coefficients:", clf.coef_
    coefs = [myround(i) for i in clf.coef_]
    # print "Rounded coefs:", coefs
    nonzero = np.nonzero(coefs)
    print nonzero
    if not len(nonzero[0]):
        return (lambda(n): 0 , "the zero function")
    idx = nonzero[0][0]
    coef = coefs[idx]
    curve_text = "$" + ("" if coef==1 else "{:}".format(coef))+descs[idx]+"$"
    
    return (lambda(n): coef*funcs[idx](n) , curve_text)

In [ ]:
def fit_exp(lengths,steps):
    EXP_MIN = 0.4
    observations = np.reshape([mylog2(n) for n in steps],(-1,1))
    lengths = np.reshape(lengths,(-1,1))
    # print "lengths, observations:", len(lengths),len(observations), lengths[:5], observations[:5]
    
    clf = linear_model.LinearRegression()
    clf.fit (lengths,observations)
    # return map(lambda c: round(c) if abs(c-round(c))<0.2 else c,cls.coefs_)
    slope = round(clf.coef_[0][0],2)    
    if slope < EXP_MIN:
        return (None,None) # this is not an exponential function
    
    scale = int(round(2**clf.intercept_))
    curve_text = "$"+ ("" if scale==1 else "{:d}".format(scale)) + r"\exp({0:g}n)".format(slope)+"$"
    return (lambda n: scale*(2**(slope*n)) , curve_text)

In [ ]:
def plot_curve(lengths,times,steps,doplot=True, doplot_logscale=True, smoothing_filter = False,interpolate=True):
    '''fit best curve to time vs input length curve'''
    plot_steps = any(steps)
    orig_steps = steps
    steps = map(lambda s: max(2,s),steps) if plot_steps else map(lambda s: int(s*(10**6)), times)
    curve_scale = sum(times)/sum(steps) if plot_steps else 10**(-6)
    times = map(lambda n: (10**6)*n, times)
    print 'plot_steps:', plot_steps
    print '{0:0.3f} micro-seconds per step'.format(curve_scale*(10**6))
    curve_text = '(none)'
    exponential = False
    EXP_SLOPE  = 0.4
    MIN_FRACTION = 0.01
    
    
    if interpolate:
        (step_curve, curve_text)= (fit_exp(lengths,steps) if plot_steps else fit_exp(lengths,times))
        if step_curve:
            exponential = True
        else:
            (step_curve,curve_text) = fit_poly_log(lengths,steps, smoothing_filter)

        print "Curve (steps):", curve_text 

        def curve(n):
            return curve_scale * step_curve(n)

        approx = [max(step_curve(n),2) for n in lengths]
    else:
        def curve(n): 
            return 1
    
    if doplot:
        fig, ax1 = plt.subplots()
        ax2 = ax1.twinx()
        ax3 = ax1.twinx()
        lns1 = ax1.plot(lengths, times, 'g-',label='time')
        if plot_steps:
            lns2 = ax2.plot(lengths, orig_steps, 'b-',label='steps')
        if interpolate:
            lns3 = ax3.plot(lengths,approx,'r--',label='curve')
        ax3.get_yaxis().set_visible(False)
        
        # added these three lines
        lns = lns1
        if plot_steps:
            lns += lns2
        if interpolate:
            lns += lns3
        labs = [l.get_label() for l in lns]
        ax1.legend(lns, labs, loc=0)

        #plt.legend()
        ax3.plot(lengths,approx,'r--')
        ax1.set_xlabel('Input length')
        ax1.set_ylabel(r'Time ($\mu$s)', color='g')
        if plot_steps:
            ax2.set_ylabel('Steps', color='b')
        
        if interpolate:
            fig.suptitle(r'Best fit: ${:0.3g} \times$'.format(curve_scale*(10**6))+curve_text) 
        plt.show()
        
        if doplot_logscale:
            fig, bx1 = plt.subplots()
            bx2 = bx1.twinx()
            bx3 = bx1.twinx()
            lns1= bx1.plot(lengths, [mylog2(n) for n in times], 'g-',label='log time')
            if plot_steps:
                lns2= bx2.plot(lengths, [mylog2(n) for n in steps], 'b-',label='log steps')
            if interpolate:
                lns3 = bx3.plot(lengths,np.log2(approx),'r--',label='log curve')
            
            lns = lns1
            if plot_steps:
                lns += lns2
            if interpolate:
                lns += lns3

            labs = [l.get_label() for l in lns]
            bx1.legend(lns, labs, loc=0)
            
            bx3.get_yaxis().set_visible(False)        
            bx1.set_xlabel('Input length')
            bx1.set_ylabel('log time', color='g')
            if plot_steps:
                bx2.set_ylabel('log steps', color='b')
            #plt.legend()
            fig.suptitle('Log scale') 
            plt.show()
        
    return curve
        
    
    

In [ ]:
def mytest(n):
    for i in range(int(n*math.log10(n))):
        step_pc()

# (o,c,_1,_2,_3) = timer(mytest,[n+1 for n in range(100)]);

# Meoization

In [ ]:
def memoized(f, return_table = False):
    '''Return a "memoized" version of the function f using internal mem_table.'''
    mem_table = dict()
    def g(*input):
        # step_pc()
        key = repr(input)
        print "Key:",key
        if key not in mem_table:
            print "not found - now value is", mem_table[key]
            mem_table[key] = f(*input)
            return mem_table[key]
        else:
            return mem_table[key]
    g.non_memoized = f
    return (g,mem_table) if return_table else g

# Graph utilities

In [ ]:
import networkx as nx

In [ ]:
def list_to_graph(L):
    # print "L=",L
    G=nx.Graph()
    for i in range(len(L)):
        G.add_node(i)
        # print "i,L[i]",i,L[i]
        for j in L[i]:
            G.add_edge(i,j)
    return G

In [ ]:
def nxpos(x,y):
    return np.transpose(np.asarray([x,y],dtype=np.float32))

In [ ]:
def grid_layout(G):
    """Takes nx.Graph object which we assume is a two dimensional grid and
       returns a layout for it.
    """
    n = nx.number_of_nodes(G)
    k = int(math.floor(math.sqrt(n)))
    _dict = {}
    for i in range(k):
        for j in range(k):
            _dict[k*i+j] = nxpos(float(i)/float(k),float(j)/float(k))
    t = n-k*k
    for u in range(t):
        _dict[k*k+u]=nxpos(1,float(u)/float(t))
    return _dict

In [ ]:
def spring100_layout(G):
        return nx.spring_layout(G,iterations=100)   

In [ ]:
def choose_layout_method(my_layout_method):
    L = [nx.shell_layout
         , spring100_layout, nx.spectral_layout]
    S = [grid_layout] # special methods - only chosen if selected by name
    M = [f for f in L+S if f.__name__ == my_layout_method]
    return (M[0] if M else random.choice(L))


In [ ]:
def draw_graph(G, my_layout_method=None):
    if not isinstance(G,nx.Graph):
        G = list_to_graph(G)
    f = choose_layout_method(my_layout_method)    
    print f.__name__
    nx.draw(G,pos=f(G))
    # plt.savefig("simple_path.png") # save as png
    plt.show() 

In [ ]:
_my.anim_graph = None 
_my.anim_layout = None
_my.anim_colors = None
_my.anim_ax = None
_my.anim_fig = None
_my.anim_layout = None
_my.anim = None

In [ ]:
def initialize_animation(G,default_color = 'w', my_layout_method = None):
    # print "Graph:", G
    _my.anim_graph = list_to_graph(G)
    # print "Graph (nx):",_my.anim_graph
    _my.anim_fig, _my.anim_ax = plt.subplots()
    f = choose_layout_method(my_layout_method)
    # print "Method =", f
    _my.anim_layout = f(_my.anim_graph)
    # print "Layout=", _my.anim_layout
    _my.anim_colors = [ [default_color for i in range(len(G))]]

def color(i,new_color):
    last_list = _my.anim_colors[-1]
    new_list = [c for c in last_list]
    new_list[i] = new_color
    _my.anim_colors.append(new_list)

def animate(i):
    nx.draw(_my.anim_graph, pos=_my.anim_layout, node_color = _my.anim_colors[i], ax = _my.anim_ax)

def show_animation(total_time = 30):
    anim = animation.FuncAnimation(_my.anim_fig, animate, 
                                   frames=len(_my.anim_colors), 
                                   interval=(total_time*1000)/len(_my.anim_colors), 
                                   blit=False, repeat=False)
    _my.anim = anim
    print 'saving..'
    anim.save('alg_viz.mp4')
    print 'rendering..'
    return HTML(anim.to_html5_video())

In [ ]:
# G = [[1],[2],[0]]
# initialize_animation(G)
# color(0,'r')
# color(1,'r')
# color(2,'r')
# show_animation()